In [2]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.17.1'

In [3]:
model = keras.models.load_model('model_2024_hairstyle.keras')


I0000 00:00:1733912823.795074   37282 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-11 13:27:03.989637: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-12-11 13:27:04.041895: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 80289792 exceeds 10% of free system memory.
2024-12-11 13:27:04.066055: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 80289792 exceeds 10% of free system memory.
2024-12-11 1

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('model_2024_hairstyle.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpzvfl9pke/assets


INFO:tensorflow:Assets written to: /tmp/tmpzvfl9pke/assets


Saved artifact at '/tmp/tmpzvfl9pke'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140507312490976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140507312495904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140507312806320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140507312808608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140507312813360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140507312814064: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1733912829.048422   37282 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1733912829.048470   37282 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-12-11 13:27:09.049415: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpzvfl9pke
2024-12-11 13:27:09.049871: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-12-11 13:27:09.049891: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpzvfl9pke
2024-12-11 13:27:09.053983: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-12-11 13:27:09.055086: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-12-11 13:27:09.262957: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpzvfl9pke
2024-12-11 13:27:09.269718: I tensorflow/cc/saved_model/loader.cc

In [5]:
import tensorflow.lite as tflite


In [7]:
interpreter = tflite.Interpreter(model_path='model_2024_hairstyle.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [8]:
print(input_index, output_index)

0 13


In [10]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [13]:
img = download_image('https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg')
img = prepare_image(img, (200, 200))

x = np.array(img)
X = np.array([x])

X = X.astype(np.float32)
X = X / 255.0

In [16]:
X

array([[[[0.23921569, 0.40784314, 0.08627451],
         [0.25490198, 0.43137255, 0.11372549],
         [0.24705882, 0.43137255, 0.1254902 ],
         ...,
         [0.23921569, 0.34117648, 0.        ],
         [0.23921569, 0.34901962, 0.01960784],
         [0.20784314, 0.3137255 , 0.00392157]],

        [[0.24313726, 0.4117647 , 0.10196079],
         [0.2509804 , 0.41960785, 0.10980392],
         [0.25490198, 0.41960785, 0.12156863],
         ...,
         [0.2509804 , 0.34117648, 0.        ],
         [0.24313726, 0.33333334, 0.01176471],
         [0.21568628, 0.32156864, 0.01960784]],

        [[0.26666668, 0.4392157 , 0.13725491],
         [0.2627451 , 0.42745098, 0.12941177],
         [0.27450982, 0.42352942, 0.12941177],
         ...,
         [0.2509804 , 0.34901962, 0.03137255],
         [0.23137255, 0.32156864, 0.01568628],
         [0.20392157, 0.29803923, 0.        ]],

        ...,

        [[0.10196079, 0.2       , 0.04313726],
         [0.10196079, 0.2       , 0.04313726]

In [14]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [15]:
preds


array([[0.8937741]], dtype=float32)